In [1]:
import numpy as np
import matplotlib.pyplot as plt
# from imblearn.metrics import sensitivity_specificity_support as sss
from sklearn.metrics import  f1_score, precision_score, recall_score, accuracy_score, roc_curve, auc, roc_auc_score, confusion_matrix as CM

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Multiply,add,concatenate,Conv3D,GlobalAveragePooling3D,  MaxPooling3D, LeakyReLU, BatchNormalization, Dropout, Flatten, Activation, Reshape,  Conv3DTranspose, UpSampling3D
from tensorflow.keras.regularizers import l2, l1, l1_l2

/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ma-user/anaconda3/envs/TensorFlow-1.13.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ma-u

In [2]:
# cretae the binary input 
def data_filter(data, label, exclu):
    idx = np.where(label!= exclu)[0]
    #print(len(idx))
    data_new = data[idx]
    label_new = label[idx]
    #print(data_new.shape)
    print(np.unique(label_new, return_counts=True))
    return data_new, label_new

# onehot encode labels for binary classifications
def onehot_bi(y):
    from sklearn.preprocessing import OneHotEncoder
    onehot_encoder = OneHotEncoder(sparse=False)
    y = y.reshape(len(y), 1)
    y_encoded = onehot_encoder.fit_transform(y)
    return y_encoded

# onehot encode labels for 3-way classifications
def onehot_tri(y):
    from keras.utils import to_categorical
    return to_categorical(y)

# view the distribution of class labels of the input data
def showpercentage(array):
    pcn = array[1][0]/np.sum(array[1])
    pmci = array[1][1]/np.sum(array[1])
    pad = array[1][2]/np.sum(array[1])
    print(str(pad) + " percent of the data has AD label")
    print(str(pcn) + " percent of the data has CN label")
    print(str(pmci) + " percent of the data has MCI label")
# visualizatio of model traning and model performance 

# visualize the training and validation performance
def plot_history(data_list, label_list, title, ylabel, name):

    epochs = range(1, len(data_list[0]) + 1)

    for data, label in zip(data_list, label_list):
        plt.plot(epochs, data, label=label)
    plt.title(title, pad = 10, fontsize='large')
    plt.xlabel('Epochs', labelpad=10)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.show()
#%%

# model evaluation
    
# evaluate model performance - binary classifications
def evaluate_binary(X_test, y_test, model, name):
    
    test_y_prob = model.predict(X_test)
    print(test_y_prob)
    test_y_pred = np.argmax(test_y_prob, axis = 1)
    test_y_true = np.argmax(y_test, axis = 1) 
    # accuracy
    
    loss, acc = model.evaluate(X_test, y_test)
    # AUC
    pos_prob = test_y_prob[:,1]
    auc_score = roc_auc_score(test_y_true, pos_prob)
    # precision, recall, specificity, and f1_score
    p = precision_score(test_y_true, test_y_pred)
    r = recall_score(test_y_true, test_y_pred)
    f1 = f1_score(test_y_true, test_y_pred)
#     sen, spe, _ = sss(test_y_true, test_y_pred, average="binary")
    print(test_y_true, test_y_pred)
    # print results
    print("Test accuracy:", acc)
    print("Test AUC is: ", auc_score)
    print("Test confusion matrix: \n", CM(test_y_true, test_y_pred))
    print("Precision: ", p)
    print("Recall: ", r)
#     print("Specificity: ", spe)
    print("f1_score: ", f1)

    # plot and save roc curve
    pos_prob = test_y_prob[:,1]
    fpr, tpr, thresholds = roc_curve(test_y_true, pos_prob)
    ns_probs = [0 for _ in range(len(test_y_prob))]
    ns_fpr, ns_tpr, _ = roc_curve(test_y_true, ns_probs)
    plt.axis([0,1,0,1]) 
    plt.plot(fpr,tpr, marker = '.', color = 'darkorange', label = 'Model AUC (area = {:.2f})'.format(auc_score)) 
    plt.plot(ns_fpr, ns_tpr, color = 'royalblue', linestyle='--')
    plt.legend()
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate') 
    plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.show()
    
# evaluate model performance - 3 way classifiations
def evaluate_3way(X_test, y_test, model):
    test_y_prob = model.predict(X_test)
    test_y_pred = np.argmax(test_y_prob, axis = 1)
    test_y_true = np.argmax(y_test, axis = 1) 
    print(test_y_prob)
    # accuracy
    loss, acc = model.evaluate(X_test, y_test)
    # precision, recall, specificity, and f1_score
    p = precision_score(test_y_true, test_y_pred, average="macro")
    r = recall_score(test_y_true, test_y_pred, average="macro")
    f1 = f1_score(test_y_true, test_y_pred, average="macro")
#     sen,spe,_ = sss(test_y_true, test_y_pred, average="macro")
    print(test_y_true, test_y_pred)
    print("Test accuracy:", acc)
    print("Test confusion matrix: \n", CM(test_y_true, test_y_pred))
    print("Precision: ", p)
    print("Recall: ", r)
#     print("Specificity: ", spe)
    print("f1_score: ", f1)

In [3]:
import logging
import numpy as np
import tensorflow as tf

In [4]:
"""  Load in the input - original data """

Xtr = np.load("preprocess/input/random_split/train_data.npy", allow_pickle = True)
ytr = np.load("preprocess/input/random_split/train_label.npy", allow_pickle = True)
print(Xtr.shape)
print(ytr.shape)

Xts = np.load("preprocess/input/random_split/test_data.npy", allow_pickle = True)
yts = np.load("preprocess/input/random_split/test_label.npy", allow_pickle = True)
print(Xts.shape)
print(yts.shape)

Xval = np.load("preprocess/input/random_split/val_data.npy", allow_pickle = True)
yval = np.load("preprocess/input/random_split/val_label.npy", allow_pickle = True)
print(Xval.shape)
print(yval.shape)

print("Train:")
showpercentage(np.unique(ytr, return_counts=True))
print()
print("Validation:")
showpercentage(np.unique(yval, return_counts=True))
print()
print("Test")
showpercentage(np.unique(yts, return_counts=True))

(356, 64, 64, 64)
(356,)
(86, 64, 64, 64)
(86,)
(90, 64, 64, 64)
(90,)
Train:
0.2893258426966292 percent of the data has AD label
0.41853932584269665 percent of the data has CN label
0.29213483146067415 percent of the data has MCI label

Validation:
0.28888888888888886 percent of the data has AD label
0.4222222222222222 percent of the data has CN label
0.28888888888888886 percent of the data has MCI label

Test
0.29069767441860467 percent of the data has AD label
0.4186046511627907 percent of the data has CN label
0.29069767441860467 percent of the data has MCI label


In [9]:
def get_densenet( X_train, y_train, X_valid = None, y_valid = None, 
             final = False, out = 2,
             dr = 0.02, lr = 0.00001, 
             breg = l2(0.0001), areg = None, 
             n_epochs = 100, batch_size = 15 ):
  
    inputs = keras.Input((64, 64, 64, 1))

    c1 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(inputs)
    
#     c1 = cbam(x)
#     c1 = add([c1, c1])
    c2 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(add([c1, c1]))
    c3 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c2)
#     c1 = add([c1, x])
    p1 = MaxPooling3D(pool_size=(2, 2, 2))(c3)
    
#     c2 = cbam(c1)
#     c3 = add([x, c2])
    c4 = add([add([p1, p1]), MaxPooling3D(pool_size=(2, 2, 2))(c1)])
    c4 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c4)
    c5 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c4)
#     c1 = add([c2, c1])
    p2 = MaxPooling3D(pool_size=(2, 2, 2))(c5)

#     c3 = cbam(c2)
#     c5 = add([x, c2])
    c6 = add([add([p2, p2]), add([MaxPooling3D(pool_size=(2, 2, 2))(p1), MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(c1))])])
    c6 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c6)
    c7 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c6)
#     c3 = add([c3, c2])
    p3 = MaxPooling3D(pool_size=(2, 2, 2))(c7)
    
#     c4 = cbam(c3)
    c8 = add([p3, p3])
    c8_1 = add([add([c8, MaxPooling3D(pool_size=(2, 2, 2))(p2)]), add([MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(p1)), MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(c1)))])])
    c8 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c8_1)
    c9 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c8)
#     c4 = add([c4, c3])
    p4 = MaxPooling3D(pool_size=(2, 2, 2))(add([c8_1, c9]))

    x = Dropout(dr)(p4)
    x = Flatten()(x)
    x = Dense(512, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    x = Dropout(dr)(x)
    x = Dense(256, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    outputs  = Dense(out, activation='softmax', activity_regularizer=areg)(x)
    
    model = keras.Model(inputs,outputs , name="3ddensenet")
    
    opt = Adam(lr = lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 

    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.5, patience = 5, 
                         verbose = 1, epsilon = 1e-4, mode = 'min')
    

    hist = model.fit(X_train, y_train,
                 batch_size = batch_size, 
                 epochs = n_epochs,
                 callbacks=[cb],
                 validation_data = (X_valid, y_valid), 
                 shuffle = True)
    
    return model, hist

In [10]:
# create input for binary classification of NC vs. AD
Xtr_ncad, ytr_ncad = data_filter(Xtr, ytr, 1)
Xval_ncad, yval_ncad = data_filter(Xval, yval, 1)
Xts_ncad, yts_ncad = data_filter(Xts, yts, 1)

# reshape the input
X_train = Xtr_ncad.reshape(-1,64,64,64,1) 
X_test = Xts_ncad.reshape(-1,64,64,64,1) 
X_val = Xval_ncad.reshape(-1,64,64,64,1) 


# one hot encode the target labels 
y_train = onehot_bi(ytr_ncad)
y_test = onehot_bi(yts_ncad)
y_val = onehot_bi(yval_ncad)



# model training
model, hist = get_densenet(X_train, y_train, X_val, y_val, 
                       breg = l2(0.001), areg = l1(0.001))

(array([0, 2]), array([149, 103]))
(array([0, 2]), array([38, 26]))
(array([0, 2]), array([36, 25]))
Train on 252 samples, validate on 64 samples
Epoch 1/100
252/252 [==============================] - 7s 29ms/sample - loss: 6.6072 - acc: 0.4643 - val_loss: 2.0455 - val_acc: 0.6406
Epoch 2/100
252/252 [==============================] - 6s 26ms/sample - loss: 3.1716 - acc: 0.5119 - val_loss: 2.7003 - val_acc: 0.6094
Epoch 3/100
252/252 [==============================] - 6s 26ms/sample - loss: 2.4556 - acc: 0.5278 - val_loss: 1.5895 - val_acc: 0.6094
Epoch 4/100
252/252 [==============================] - 6s 26ms/sample - loss: 1.8535 - acc: 0.6389 - val_loss: 0.7993 - val_acc: 0.5938
Epoch 5/100
252/252 [==============================] - 6s 26ms/sample - loss: 1.6722 - acc: 0.6349 - val_loss: 0.7841 - val_acc: 0.6250
Epoch 6/100
252/252 [==============================] - 6s 26ms/sample - loss: 1.5300 - acc: 0.5595 - val_loss: 1.0404 - val_acc: 0.6562
Epoch 7/100
252/252 [=================

In [ ]:
def get_cnn(out = 2,
             dr = 0.02, lr = 0.00001, 
             breg = l2(0.0001), areg = None):
    inputs = keras.Input((64, 64, 64, 1))

    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(inputs)
    
#     c1 = cbam(x)
    c1 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    c1 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c1)
#     c1 = add([c1, x])
    c1 = MaxPooling3D(pool_size=(2, 2, 2))(c1)
    
#     c2 = cbam(c1)
    c2 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c1)
    c2 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c2)
#     c1 = add([c2, c1])
    c2 = MaxPooling3D(pool_size=(2, 2, 2))(c2)

#     c3 = cbam(c2)
    c3 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c2)
    c3 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c3)
#     c3 = add([c3, c2])
    c3 = MaxPooling3D(pool_size=(2, 2, 2))(c3)
    
#     c4 = cbam(c3)
    c4 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c3)
    c4 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c4)
#     c4 = add([c4, c3])
    c4 = MaxPooling3D(pool_size=(2, 2, 2))(c4)

    x = Dropout(dr)(c4)
    x = Flatten()(x)
    x = Dense(512, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    x = Dropout(dr)(x)
    x = Dense(256, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    outputs  = Dense(out, activation='softmax', activity_regularizer=areg)(x)
    
    model = keras.Model(inputs,outputs , name="3dcnn")
    return model

In [40]:
def get_dense(out = 2,
             dr = 0.02, lr = 0.00001, 
             breg = l2(0.0001), areg = None):
    inputs = keras.Input((64, 64, 64, 1))

    c1 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(inputs)
    
#     c1 = cbam(x)
#     c1 = add([c1, c1])
    c2 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(add([c1, c1]))
    c3 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c2)
#     c1 = add([c1, x])
    p1 = MaxPooling3D(pool_size=(2, 2, 2))(c3)
    
#     c2 = cbam(c1)
#     c3 = add([x, c2])
    c4 = add([add([p1, p1]), MaxPooling3D(pool_size=(2, 2, 2))(c1)])
    c4 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c4)
    c5 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c4)
#     c1 = add([c2, c1])
    p2 = MaxPooling3D(pool_size=(2, 2, 2))(c5)

#     c3 = cbam(c2)
#     c5 = add([x, c2])
    c6 = add([add([p2, p2]), add([MaxPooling3D(pool_size=(2, 2, 2))(p1), MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(c1))])])
    c6 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c6)
    c7 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c6)
#     c3 = add([c3, c2])
    p3 = MaxPooling3D(pool_size=(2, 2, 2))(c7)
    
#     c4 = cbam(c3)
    c8 = add([p3, p3])
    c8_1 = add([add([c8, MaxPooling3D(pool_size=(2, 2, 2))(p2)]), add([MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(p1)), MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(MaxPooling3D(pool_size=(2, 2, 2))(c1)))])])
    c8 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c8_1)
    c9 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c8)
#     c4 = add([c4, c3])
    p4 = MaxPooling3D(pool_size=(2, 2, 2))(c9)

    x = Dropout(dr)(p4)
    x = Flatten()(x)
    x = Dense(512, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    x = Dropout(dr)(x)
    x = Dense(256, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    outputs  = Dense(out, activation='softmax', activity_regularizer=areg)(x)
    
    model = keras.Model(inputs,outputs , name="3dcnn")
    return model

In [41]:
model=get_dense()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 64, 64, 64, 1 0                                            
__________________________________________________________________________________________________
conv3d_127 (Conv3D)             (None, 64, 64, 64, 3 896         input_18[0][0]                   
__________________________________________________________________________________________________
add_53 (Add)                    (None, 64, 64, 64, 3 0           conv3d_127[0][0]                 
                                                                 conv3d_127[0][0]                 
__________________________________________________________________________________________________
conv3d_128 (Conv3D)             (None, 64, 64, 64, 3 27680       add_53[0][0]                     
__________

In [7]:
def get_resnet( X_train, y_train, X_valid = None, y_valid = None, 
             final = False, out = 2,
             dr = 0.02, lr = 0.00001, 
             breg = l2(0.0001), areg = None, 
             n_epochs = 100, batch_size = 15 ):
  
    inputs = keras.Input((64, 64, 64, 1))

    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(inputs)

    c1 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    c1_1 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c1)
    c1_1 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c1_1)
    c1_1 = add([c1, c1_1])
    c1_1 = MaxPooling3D(pool_size=(2, 2, 2))(c1_1)
    
    c2 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c1)
    c2_2 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c2)
    c2_2 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c2_2)
    c2_2 = add([c2, c2_2])
    c2_2 = MaxPooling3D(pool_size=(2, 2, 2))(c2_2)

    c3 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c2)
    c3_3 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c3)
    c3_3 = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c3_3)
    c3_3 = add([c3, c3_3])
    c3_3 = MaxPooling3D(pool_size=(2, 2, 2))(c3_3)

    c4 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c3)
    c4_4 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c4)
    c4_4 = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(c4_4)
    c4_4 = add([c4, c4_4])
    c4_4 = MaxPooling3D(pool_size=(2, 2, 2))(c4_4)

    x = Dropout(dr)(c4_4)
    x = Flatten()(x)
    x = Dense(512, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    x = Dropout(dr)(x)
    x = Dense(256, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    outputs  = Dense(out, activation='softmax', activity_regularizer=areg)(x)
    
    model = keras.Model(inputs,outputs , name="3dresnet")
    
    opt = Adam(lr = lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 

    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.5, patience = 5, 
                         verbose = 1, epsilon = 1e-4, mode = 'min')
    

    hist = model.fit(X_train, y_train,
                 batch_size = batch_size, 
                 epochs = n_epochs,
                 callbacks=[cb],
                 validation_data = (X_valid, y_valid), 
                 shuffle = True)
    
    return model, hist

In [8]:
# create input for binary classification of NC vs. AD
Xtr_ncad, ytr_ncad = data_filter(Xtr, ytr, 1)
Xval_ncad, yval_ncad = data_filter(Xval, yval, 1)
Xts_ncad, yts_ncad = data_filter(Xts, yts, 1)

# reshape the input
X_train = Xtr_ncad.reshape(-1,64,64,64,1) 
X_test = Xts_ncad.reshape(-1,64,64,64,1) 
X_val = Xval_ncad.reshape(-1,64,64,64,1) 


# one hot encode the target labels 
y_train = onehot_bi(ytr_ncad)
y_test = onehot_bi(yts_ncad)
y_val = onehot_bi(yval_ncad)



# model training
model, hist = get_resnet(X_train, y_train, X_val, y_val, 
                       breg = l2(0.001), areg = l1(0.001))

(array([0, 2]), array([149, 103]))
(array([0, 2]), array([38, 26]))
(array([0, 2]), array([36, 25]))
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 252 samples, validate on 64 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
252/252 [==============================] - 20s 78ms/sample - loss: 3.4064 - acc: 0.5357 - val_loss: 0.6118 - val_acc: 0.6719
Epoch 2/100
252/252 [==============================] - 13s 53ms/sample - loss: 0.6717 - acc: 0.6667 - val_loss: 0.4549 - val_acc: 0.8594
Epoch 3/100
252/252 [==============================] - 13s 53ms/sample - loss: 0.7113 - acc: 0.6944 - val_loss: 0.5008 - val_acc: 0.7188
Epoch 4/100
252/252 [==============================] - 13s 53ms/sample - loss: 0.3961 - acc: 0.8294 - val_loss: 0.6821 - val_acc: 0.5469
Epoch 5/100
252/252 [==============================] - 13s 53ms/sample 

KeyboardInterrupt: 

In [9]:
model=get_cnn()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 64, 1)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 64, 64, 64, 32)    896       
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 64, 64, 64, 32)    27680     
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 64, 64, 64, 32)    27680     
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 32, 32, 32, 32)    0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 32, 32, 32, 32)    27680     
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 32, 32, 32, 32)    27680     
__________

In [5]:
def cbam(inputs):
    inputs_channels=int(inputs.shape[-1])
    x=GlobalAveragePooling3D()(inputs)
    x=Dense(int(inputs_channels/4),activation="relu")(x)
    x=Dense(int(inputs_channels),activation="softmax")(x)
    x=Reshape((1,1,1,inputs_channels))(x)
    x=Multiply()([inputs,x])
    return x
def get_voxcnn( X_train, y_train, X_valid = None, y_valid = None, 
             final = False, out = 2,
             dr = 0.02, lr = 0.00001, 
             breg = l2(0.0001), areg = None, 
             n_epochs = 100, batch_size = 15 ):
  
    inputs = keras.Input((64, 64, 64, 1))

    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(inputs)
    
    x = cbam(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    x = cbam(x)
    x = Conv3D(64, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(64, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    x = cbam(x)
    x = Conv3D(128, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(128, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
    x = cbam(x)
    x = Conv3D(256, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(256, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = Dropout(dr)(x)
    x = Flatten()(x)
    x = Dense(512, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    x = Dropout(dr)(x)
    x = Dense(256, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    outputs  = Dense(out, activation='softmax', activity_regularizer=areg)(x)
    
    model = keras.Model(inputs,outputs , name="3dcnn")
    
    opt = Adam(lr = lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 

    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.5, patience = 5, 
                         verbose = 1, epsilon = 1e-4, mode = 'min')
    

    hist = model.fit(X_train, y_train,
                 batch_size = batch_size, 
                 epochs = n_epochs,
                 callbacks=[cb],
                 validation_data = (X_valid, y_valid), 
                 shuffle = True)
    
    return model, hist

In [12]:
def get_cnn( X_train, y_train, X_valid = None, y_valid = None, 
             final = False, out = 2,
             dr = 0.02, lr = 0.00001, 
             breg = l2(0.0001), areg = None, 
             n_epochs = 100, batch_size = 15 ):
  
    inputs = keras.Input((64, 64, 64, 1))

    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(inputs)
    
#     x = cbam(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
#     x = cbam(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
#     x = cbam(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(32, kernel_size=(3,3,3),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)
    
#     x = cbam(x)
    x = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = Conv3D(32, kernel_size=(2,2,2),  kernel_initializer='he_uniform', padding="same",activation="relu", bias_regularizer=breg)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = Dropout(dr)(x)
    x = Flatten()(x)
    x = Dense(512, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    x = Dropout(dr)(x)
    x = Dense(256, bias_regularizer=breg,   kernel_initializer='he_uniform',activation="relu")(x)
    outputs  = Dense(out, activation='softmax', activity_regularizer=areg)(x)
    
    model = keras.Model(inputs,outputs , name="3dcnn")
    
    opt = Adam(lr = lr)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy']) 

    cb = ReduceLROnPlateau(monitor = 'val_loss', 
                         factor = 0.5, patience = 5, 
                         verbose = 1, epsilon = 1e-4, mode = 'min')
    

    hist = model.fit(X_train, y_train,
                 batch_size = batch_size, 
                 epochs = n_epochs,
                 callbacks=[cb],
                 validation_data = (X_valid, y_valid), 
                 shuffle = True)
    
    return model, hist

In [ ]:
# create input for binary classification of NC vs. AD
Xtr_ncad, ytr_ncad = data_filter(Xtr, ytr, 1)
Xval_ncad, yval_ncad = data_filter(Xval, yval, 1)
Xts_ncad, yts_ncad = data_filter(Xts, yts, 1)

# reshape the input
X_train = Xtr_ncad.reshape(-1,64,64,64,1) 
X_test = Xts_ncad.reshape(-1,64,64,64,1) 
X_val = Xval_ncad.reshape(-1,64,64,64,1) 


# one hot encode the target labels 
y_train = onehot_bi(ytr_ncad)
y_test = onehot_bi(yts_ncad)
y_val = onehot_bi(yval_ncad)



# model training
model, hist = get_cnn(X_train, y_train, X_val, y_val, 
                       breg = l2(0.001), areg = l1(0.001))

(array([0, 2]), array([149, 103]))
(array([0, 2]), array([38, 26]))
(array([0, 2]), array([36, 25]))
Train on 252 samples, validate on 64 samples
Epoch 1/100
252/252 [==============================] - 7s 27ms/sample - loss: 0.7652 - acc: 0.5119 - val_loss: 0.6828 - val_acc: 0.5938
Epoch 2/100
252/252 [==============================] - 6s 24ms/sample - loss: 0.7071 - acc: 0.5952 - val_loss: 0.6657 - val_acc: 0.5938
Epoch 3/100
252/252 [==============================] - 6s 24ms/sample - loss: 0.6809 - acc: 0.6071 - val_loss: 0.6599 - val_acc: 0.5938
Epoch 4/100
252/252 [==============================] - 6s 24ms/sample - loss: 0.6551 - acc: 0.6151 - val_loss: 0.6529 - val_acc: 0.5938
Epoch 5/100
252/252 [==============================] - 6s 24ms/sample - loss: 0.6554 - acc: 0.6032 - val_loss: 0.6484 - val_acc: 0.5938
Epoch 6/100
252/252 [==============================] - 6s 24ms/sample - loss: 0.6361 - acc: 0.6468 - val_loss: 0.6394 - val_acc: 0.5938
Epoch 7/100
252/252 [=================